In [1]:
from transformers import AutoTokenizer

#加载字典和分词工具,这里的checkpoint一般和预训练模型保持一致,两者是成对使用的
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

#试编码一句话,所有一句编码后都是等长的
out = tokenizer(text='测试语句',
                padding='max_length',
                max_length=8,
                return_tensors='pt',
                truncation=True)

#查看编码结果
for k, v in out.items():
    print(k, v, v.shape, v.dtype)

#反编码
print(tokenizer.decode(out['input_ids'][0]))

/root/anaconda3/envs/pt2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


input_ids tensor([[ 101, 3844, 6407, 6427, 1368,  102,    0,    0]]) torch.Size([1, 8]) torch.int64
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0]]) torch.Size([1, 8]) torch.int64
attention_mask tensor([[1, 1, 1, 1, 1, 1, 0, 0]]) torch.Size([1, 8]) torch.int64
[CLS] 测 试 语 句 [SEP] [PAD] [PAD]


In [2]:
from datasets import load_dataset, load_from_disk

#在线加载数据集,国内网络环境得看运气
#更多数据集:https://huggingface.co/datasets
#dataset = load_dataset(path='lansinuote/ChnSentiCorp')

#保存到磁盘
#dataset.save_to_disk('data/ChnSentiCorp')

#加载离线数据集
dataset = load_from_disk('data/ChnSentiCorp')

dataset, dataset['train'][0]

(DatasetDict({
     validation: Dataset({
         features: ['text', 'label'],
         num_rows: 1200
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 1200
     })
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 9600
     })
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  'label': 1})

In [3]:
#添加取数据时的回调函数,可以在这里完成数据的处理工作
def f(data):
    #编码句子
    result = tokenizer(text=data['text'],
                       padding='max_length',
                       max_length=200,
                       return_tensors='pt',
                       truncation=True)

    #label字段不能落下,添加到返回结果里
    result['label'] = data['label']

    return result


dataset = dataset.with_transform(f, output_all_columns=False)

dataset['train'][0]

{'input_ids': tensor([ 101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221,
         3175,  912, 8024, 3300, 4510, 1220, 2820, 3461, 4684, 2970, 1168, 6809,
         3862, 6804, 8024, 1453, 1741, 7623, 7667,  510, 7608, 2443,  510, 1555,
         1767,  510, 6631, 2356,  510, 3033,  855,  671, 2418,  936, 1059,  511,
         6983, 2421, 6163,  934,  671, 5663, 8024,  852, 6820, 5050, 3146, 3815,
          511, 3807, 3737, 1762, 1920, 1828, 4638, 2238, 7553, 8024, 1728, 3634,
         2523, 2207, 8024,  679, 6814, 1957, 1036,  948, 3221, 1599, 3614,  511,
         1259, 4638, 3193, 7623, 3221, 6205, 2466, 4638, 8024, 6820, 5050,  705,
         2168,  511, 3302, 1218, 1408, 8024,  671, 5663,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

In [4]:
import torch

loader = torch.utils.data.DataLoader(dataset=dataset['train'],
                                     batch_size=8,
                                     shuffle=True,
                                     drop_last=True)

for k, v in next(iter(loader)).items():
    print(k, v.shape, v.dtype)

len(loader)

input_ids torch.Size([8, 200]) torch.int64
token_type_ids torch.Size([8, 200]) torch.int64
attention_mask torch.Size([8, 200]) torch.int64
label torch.Size([8]) torch.int64


1200

In [5]:
from transformers import AutoModelForSequenceClassification


#手动定义模型
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()

        #加载预训练模型,自动添加分类头,可以直接使用该模型训练
        #在线加载的方式,国内的网络得看运气
        #更多模型:https://huggingface.co/models
        #pretrained = AutoModelForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=2)

        #保存到磁盘
        #pretrained.save_pretrained('model/9.pretrained.model')

        #离线加载
        pretrained = AutoModelForSequenceClassification.from_pretrained(
            './model/9.pretrained.model', num_labels=2)

        #锁定参数,不训练
        for param in pretrained.parameters():
            param.requires_grad_(False)

        #只需要特征抽取部分
        self.pretrained = pretrained.bert
        self.pretrained.eval()

        #线性输出层,这部分是要重新训练的
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, data):
        #调用预训练模型抽取参数,因为预训练模型是不训练的,所以这里不需要计算梯度
        with torch.no_grad():
            #[8, 200, 768]
            out = self.pretrained(
                input_ids=data['input_ids'],
                token_type_ids=data['token_type_ids'],
                attention_mask=data['attention_mask']).last_hidden_state

        #只要取第0个字符的特征计算分类结果即可,这和BERT模型本身的训练方式有关系
        #[8, 200, 768] -> [8, 768]
        out = out[:, 0]

        #计算线性输出
        #[8, 768] -> [8, 2]
        return self.fc(out)


model = Model()

#试算
model({
    'input_ids': torch.ones(8, 200).long(),
    'token_type_ids': torch.ones(8, 200).long(),
    'attention_mask': torch.ones(8, 200).long(),
    'label': torch.ones(8).long(),
}).shape

torch.Size([8, 2])

In [6]:
#训练
def train():
    #注意这里的参数列表,只包括要训练的参数即可
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)
    loss_fun = torch.nn.CrossEntropyLoss()
    model.fc.train()

    #定义计算设备,优先使用gpu
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    print('device=', device)

    for i, data in enumerate(loader):
        #如果使用gpu,数据要搬运到显存里
        for k in data.keys():
            data[k] = data[k].to(device)

        out = model(data)
        loss = loss_fun(out, data['label'])

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 200 == 0:
            acc = (out.argmax(dim=1) == data['label']).sum().item() / len(
                data['label'])
            print(i, loss.item(), acc)

    #保存模型,只保存训练的部分即可
    torch.save(model.fc.to('cpu'), 'model/9.model')


train()

device= cpu
0 0.617880642414093 0.625
200 0.7409070134162903 0.5
400 0.12331488728523254 0.875
600 0.1410677433013916 1.0
800 0.162505105137825 0.875
1000 0.10886094719171524 1.0


In [7]:
#测试
@torch.no_grad()
def test():

    #加载保存的模型
    model.fc = torch.load('model/9.model')
    model.fc.eval()

    #加载测试数据集,共10000条数据
    loader_test = torch.utils.data.DataLoader(dataset=dataset['test'],
                                              batch_size=8,
                                              shuffle=True,
                                              drop_last=True)

    correct = 0
    total = 0
    for i in range(100):
        data = next(iter(loader_test))

        #这里因为数据量不大,使用cpu计算就可以了
        out = model(data).argmax(dim=1)

        correct += (out == data['label']).sum().item()
        total += len(data['label'])

    print(correct / total)


test()

0.90375
